Importing dependencies


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import PIL as pillow
from PIL import Image, ImageOps
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
# print(len(gpus))

In [5]:
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

Verifying image format

In [6]:
f = open('train_labels.txt', 'r')
dir = 'data/'
train_data = f.read()
train_data = train_data.split('\n')
train_labels = []
train_images = []
for line in train_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        train_images.append(img)
        y = int(line[1])  
        train_labels.append(y)

In [7]:
f = open('validation_labels.txt', 'r')
dir = 'data/'
val_data = f.read()
val_data = val_data.split('\n')
val_labels = []
val_images = []
for line in val_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        val_images.append(img)
        y = int(line[1])  
        val_labels.append(y)

In [8]:
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [9]:
model = Sequential()
# model.add(Conv2D(128, (3,3), 1, activation='relu', input_shape=(224,224,1)))
# model.add(MaxPooling2D(2,2))
# model.add(Dropout(0.4))
model.add(Conv2D(64, (3,3), 1, activation='relu', input_shape=(224,224,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [13]:
cnn = model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
235/235 [==============================] - 804s 3s/step - loss: 0.3332 - accuracy: 0.8508
Epoch 2/5
235/235 [==============================] - 750s 3s/step - loss: 0.3227 - accuracy: 0.8587
Epoch 3/5
235/235 [==============================] - 747s 3s/step - loss: 0.3095 - accuracy: 0.8700
Epoch 4/5
235/235 [==============================] - 721s 3s/step - loss: 0.2983 - accuracy: 0.8727
Epoch 5/5
235/235 [==============================] - 835s 4s/step - loss: 0.2811 - accuracy: 0.8795


In [14]:
f = open('sample_submission.txt', 'r')
dir = 'data/'
submission_data = f.read()
submission_data = submission_data.split('\n')
sub = open('submission.csv', 'w')
sub.write('id,class\n')

for line in submission_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        res = model.predict(np.array([img]))[0][0]
        if res > 0.1492:
            res = 1
        else:
            res = 0
        sub.write(x + ',' + str(res) + '\n')

sub.close()

1/1 [==============================] - 0s 37ms/step
